# **ETL - Extract, Transform, Load Process**

## Import required libraries 

In [ ]:
# This cell imports essential libraries for data manipulation (`pandas`, `numpy`), visualization (`matplotlib`, `seaborn`), and sets display options for better readability.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Optional settings for better display
pd.set_option('display.max_columns', None)
sns.set_theme(style='whitegrid')

> **Legend**  
> `[AI-ASSIST: ChatGPT 06/08/2025]` – cell drafted/refactored with AI; author reviewed.  
> `[AI-ASSIST: Copilot 06/08/2025]` – inline suggestion/bugfix; validated by author. 

## Objectives

* This notebook performs the Extract, Transform, Load (ETL) process for the Diabetes dataset.
We aim to load all raw CSV files, clean and transform the data where necessary, and prepare a combined dataset for analysis.

- Extract multiple diabetes-related datasets from CSV files.
- Merge these datasets into a single, unified DataFrame for streamlined analysis.
- Clean and preprocess the combined data to prepare it for exploratory analysis and modeling.
- Maintain reproducibility by properly managing working directories and file paths.

## Inputs

This project uses three related CSV files containing health indicators and diabetes classification data from the 2015 Behavioral Risk Factor Surveillance System (BRFSS). Each file represents a slightly different version of the dataset for classification analysis.

* `diabetes_012_health_indicators_BRFSS2015.csv`: Multiclass target diabetes data from the 2015 Behavioral Risk Factor Surveillance System (BRFSS).
- `diabetes_binary_5050split_health_indicators_BRFSS2015.csv`: Balanced binary-class dataset, 50/50 split of diabetic and non-diabetic cases.
- `diabetes_binary_health_indicators_BRFSS2015.csv`: Original binary classification dataset with an imbalanced class distribution.

Location: All files are in the `data/` folder.

## Outputs

* A combined DataFrame merging all three datasets.
- A cleaned CSV file (`combined_diabetes_data.csv`) saved for further use.
- Prepared data ready for exploratory data analysis (EDA) and machine learning modeling.

## Additional Comments

* All transformations will be documented and commented during each step. Column names and values will be standardised for consistency.
- The three datasets vary slightly in class distribution and labeling, so adding a `source` column helps track their origin after merging.
- Working directory adjustments ensure file paths remain consistent regardless of notebook location.
- This notebook focuses on the ETL (Extract, Transform, Load) process; subsequent notebooks will handle EDA and modeling.
- Dependencies such as pandas and os are used for data handling and file operations.



# Change working directory

We are ensuring the working directory is set to the project root so file paths (e.g. for reading data) work reliably.

* We are assuming the notebooks will be stored in a subfolder, therefore when running the notebook in the editor, we will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [25]:
# This cell ensures the notebook is running from the project root so file paths work correctly. It changes the working directory to the parent folder of the notebook.
import os

# Change working directory to the project root (one level up from 'notebooks')
os.chdir('..')
print("Working directory set to:", os.getcwd())

Working directory set to: /Users/nasraibrahim/Documents/vscode-projects


In [26]:
# This cell prints the current working directory and lists files in the data folder to confirm setup.
import os
current_dir = os.getcwd()
current_dir

'/Users/nasraibrahim/Documents/vscode-projects'

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [27]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


Confirm the new current directory

In [28]:
current_dir = os.getcwd()
current_dir

'/Users/nasraibrahim/Documents'

In [29]:
import os
print("Current working directory:", os.getcwd())

Current working directory: /Users/nasraibrahim/Documents


In [30]:
import os

# Set this path to your actual project folder
project_root = "/Users/nasraibrahim/Documents/vscode-projects/diabetes-data-analysis"

os.chdir(project_root)
print("New working directory:", os.getcwd())

New working directory: /Users/nasraibrahim/Documents/vscode-projects/diabetes-data-analysis


In [31]:
# Confirm working directory and list files
import os

print("Current working directory:", os.getcwd())
print("Files in data folder:", os.listdir('data'))

Current working directory: /Users/nasraibrahim/Documents/vscode-projects/diabetes-data-analysis
Files in data folder: ['diabetes_012_health_indicators_BRFSS2015.csv', 'diabetes_binary_health_indicators_BRFSS2015.csv', 'diabetes_binary_5050split_health_indicators_BRFSS2015.csv']


# Section 1- Extract the Data

Section 1 content- 

In this section, we load the three diabetes-related CSV files stored in the `data` folder.  
We first verify that all files exist, then read each into a Pandas DataFrame.  
To keep track of the source of each record, a new column `source` is added to each DataFrame.  
Finally, we combine all three datasets into a single DataFrame called `combined_df` for further processing.

In [32]:
import pandas as pd

In [ ]:
# This cell checks if each CSV file exists, loads them into DataFrames, adds a source column to track origin, and merges them into one DataFrame for analysis.
import pandas as pd
import os

current_dir = os.getcwd()  # define current_dir first
print("Current working directory:", os.getcwd())

# File paths (use absolute paths based on current_dir)
file_1 = os.path.join(current_dir, "data/diabetes_012_health_indicators_BRFSS2015.csv")
file_2 = os.path.join(current_dir, "data/diabetes_binary_5050split_health_indicators_BRFSS2015.csv")
file_3 = os.path.join(current_dir, "data/diabetes_binary_health_indicators_BRFSS2015.csv")

# Check each file exists
for f in [file_1, file_2, file_3]:
    if not os.path.exists(f):
        raise FileNotFoundError(f"File not found: {f}")

# Load CSVs
df1 = pd.read_csv(file_1)
df2 = pd.read_csv(file_2)
df3 = pd.read_csv(file_3)

# Optional: Add a source column to keep track of origin
df1['source'] = 'original'
df2['source'] = 'balanced_5050'
df3['source'] = 'binary'

# Merge
combined_df = pd.concat([df1, df2, df3], ignore_index=True)

# Preview
combined_df.head()
# AI-ASSIST: ChatGPT + Copilot (06/08/2025) — draft + refactor; author reviewed and tested.


Current working directory: /Users/nasraibrahim/Documents/vscode-projects/diabetes-data-analysis


,Diabetes_012,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,...,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income,source,Diabetes_binary
0,0.0,1.0,1.0,1.0,40.0,1.0,0.0,0.0,0.0,0.0,...,5.0,18.0,15.0,1.0,0.0,9.0,4.0,3.0,original,NaN
1,0.0,0.0,0.0,0.0,25.0,1.0,0.0,0.0,1.0,0.0,...,3.0,0.0,0.0,0.0,0.0,7.0,6.0,1.0,original,NaN
2,0.0,1.0,1.0,1.0,28.0,0.0,0.0,0.0,0.0,1.0,...,5.0,30.0,30.0,1.0,0.0,9.0,4.0,8.0,original,NaN
3,0.0,1.0,0.0,1.0,27.0,0.0,0.0,0.0,1.0,1.0,...,2.0,0.0,0.0,0.0,0.0,11.0,3.0,6.0,original,NaN
4,0.0,1.0,1.0,1.0,24.0,0.0,0.0,0.0,1.0,1.0,...,2.0,3.0,0.0,0.0,0.0,11.0,5.0,4.0,original,NaN


---

# Section 2- Transform the Data

Section 2 content

In this section, we perform necessary data cleaning and transformation steps on the combined dataset.  
This includes handling missing values, converting data types if needed, and possibly creating new features.  
The goal is to prepare a clean and consistent dataset for analysis.

In [ ]:
# This cell checks for missing values, fills them (example for Diabetes_binary), prints data types, removes duplicate rows, and previews the cleaned data.
# Check for missing values
missing_counts = combined_df.isnull().sum()
print("Missing values per column:\n", missing_counts)

# Example: Fill missing values if appropriate (e.g., with mode or median)
# Here we fill missing values in 'Diabetes_binary' with mode as example
if combined_df['Diabetes_binary'].isnull().any():
    mode_value = combined_df['Diabetes_binary'].mode()[0]
    combined_df['Diabetes_binary'].fillna(mode_value, inplace=True)

# Convert data types if needed (example)
# combined_df['Age'] = combined_df['Age'].astype(int)

# Check data types
print("Data types:\n", combined_df.dtypes)

# Optionally drop duplicate rows if any
before_dedup = combined_df.shape[0]
combined_df.drop_duplicates(inplace=True)
after_dedup = combined_df.shape[0]
print(f"Dropped {before_dedup - after_dedup} duplicate rows.")

# Preview cleaned data
combined_df.head()
# AI-ASSIST: ChatGPT + Copilot (06/08/2025) — draft + refactor; author reviewed and tested.

Missing values per column:
 Diabetes_012            324372
HighBP                       0
HighChol                     0
CholCheck                    0
BMI                          0
Smoker                       0
Stroke                       0
HeartDiseaseorAttack         0
PhysActivity                 0
Fruits                       0
Veggies                      0
HvyAlcoholConsump            0
AnyHealthcare                0
NoDocbcCost                  0
GenHlth                      0
MentHlth                     0
PhysHlth                     0
DiffWalk                     0
Sex                          0
Age                          0
Education                    0
Income                       0
source                       0
Diabetes_binary         253680
dtype: int64
Data types:
 Diabetes_012            float64
HighBP                  float64
HighChol                float64
CholCheck               float64
BMI                     float64
Smoker                  float64
Stroke    

,Diabetes_012,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,...,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income,source,Diabetes_binary
0,0.0,1.0,1.0,1.0,40.0,1.0,0.0,0.0,0.0,0.0,...,5.0,18.0,15.0,1.0,0.0,9.0,4.0,3.0,original,0.0
1,0.0,0.0,0.0,0.0,25.0,1.0,0.0,0.0,1.0,0.0,...,3.0,0.0,0.0,0.0,0.0,7.0,6.0,1.0,original,0.0
2,0.0,1.0,1.0,1.0,28.0,0.0,0.0,0.0,0.0,1.0,...,5.0,30.0,30.0,1.0,0.0,9.0,4.0,8.0,original,0.0
3,0.0,1.0,0.0,1.0,27.0,0.0,0.0,0.0,1.0,1.0,...,2.0,0.0,0.0,0.0,0.0,11.0,3.0,6.0,original,0.0
4,0.0,1.0,1.0,1.0,24.0,0.0,0.0,0.0,1.0,1.0,...,2.0,3.0,0.0,0.0,0.0,11.0,5.0,4.0,original,0.0


In this stage of the ETL process, we performed essential data cleaning and transformation tasks to ensure the dataset is prepared for further analysis and modeling.

✅ Actions Taken:
Missing Value Analysis:

We identified a significant number of missing values in two target-related columns:

Diabetes_012: 324,372 missing values

Diabetes_binary: 253,680 missing values

No missing values were found in the remaining features.

Duplicate Handling:

A total of 49,740 duplicate rows were detected and removed from the dataset to ensure the integrity and uniqueness of the data.

🔍 Observations:
The missing values in Diabetes_012 and Diabetes_binary likely originate from how the datasets were merged — not all source files contained both target columns.

All other health indicator features (such as HighBP, CholCheck, BMI, etc.) are fully populated and can be reliably used for further processing.

The source column has been preserved to indicate the origin of each record, which may be useful for stratified analysis or understanding data distribution.

# Section 3 - Data Cleaning: Handling Missing Values and Duplicates

In this step, we address common data quality issues:

1. **Missing values** — These can lead to incorrect model training or analysis.
    - We use different strategies to fill them:
        - **Mean**: Useful for normally distributed numerical data.
        - **Median**: Best for skewed numerical data (less sensitive to outliers).
        - **Mode**: Best for categorical or ordinal features.
2. **Duplicates** — Repeated rows can bias the results and should be removed.

We'll apply these methods and save a fully cleaned version of our combined dataset.

In [ ]:
# This cell fills missing values in target columns using median/mode and removes duplicates, then prints the results to confirm the cleaning process.
import pandas as pd

# Display missing values before cleaning
print("Missing values per column BEFORE cleaning:\n", combined_df.isnull().sum())

# --- Handle missing values ---
# Fill numeric column 'Diabetes_012' with median (skewed distribution)
combined_df["Diabetes_012"] = combined_df["Diabetes_012"].fillna(combined_df["Diabetes_012"].median())

# Fill 'Diabetes_binary' with mode (binary classification target)
combined_df["Diabetes_binary"] = combined_df["Diabetes_binary"].fillna(combined_df["Diabetes_binary"].mode()[0])

# Example: If you had a feature like 'Age' with missing values, you could use mean
# combined_df["Age"] = combined_df["Age"].fillna(combined_df["Age"].mean())

# --- Remove duplicate rows ---
before_dupes = combined_df.shape[0]
combined_df = combined_df.drop_duplicates()
after_dupes = combined_df.shape[0]
print(f"\n✅ Dropped {before_dupes - after_dupes} duplicate rows.")

# Check missing values after cleaning
print("\nMissing values per column AFTER cleaning:\n", combined_df.isnull().sum())
# AI-ASSIST: ChatGPT + Copilot (06/08/2025) — draft + refactor; author reviewed and tested.

Missing values per column BEFORE cleaning:
 Diabetes_012            298531
HighBP                       0
HighChol                     0
CholCheck                    0
BMI                          0
Smoker                       0
Stroke                       0
HeartDiseaseorAttack         0
PhysActivity                 0
Fruits                       0
Veggies                      0
HvyAlcoholConsump            0
AnyHealthcare                0
NoDocbcCost                  0
GenHlth                      0
MentHlth                     0
PhysHlth                     0
DiffWalk                     0
Sex                          0
Age                          0
Education                    0
Income                       0
source                       0
Diabetes_binary              0
dtype: int64

✅ Dropped 0 duplicate rows.

Missing values per column AFTER cleaning:
 Diabetes_012            0
HighBP                  0
HighChol                0
CholCheck               0
BMI                   

# Section 4- Load the Data

In this step, we performed essential data cleaning tasks to prepare the dataset for analysis:

Removed duplicate rows to ensure there are no redundant records that could bias the results.

Handled missing values by imputing appropriate values depending on the data type:

Used mean or median for numerical columns.

Used mode for categorical columns.

After cleaning, we validated the dataset by checking for any remaining missing values and duplicates to confirm that the data is now clean.

Finally, the cleaned and validated dataset was saved as a new CSV file (combined_cleaned_final.csv) to preserve the cleaned state and avoid confusion with previous versions.

---

In [41]:
# This cell checks for any remaining missing values and duplicates, then saves the cleaned DataFrame to a new CSV file for downstream use.
# Check for any remaining missing values per column
missing_after = combined_df.isnull().sum()
print("Missing values after cleaning:")
print(missing_after[missing_after > 0])  # Show only columns with missing values

# Check for duplicates after dropping
duplicates_after = combined_df.duplicated().sum()
print(f"Number of duplicate rows remaining after dropping: {duplicates_after}")

Missing values after cleaning:
Series([], dtype: int64)
Number of duplicate rows remaining after dropping: 0


In [ ]:
# Define output file path for cleaned data
output_file = "data/combined_cleaned_final.csv"

# Ensure combined_df exists
if 'combined_df' not in locals():
    raise NameError("combined_df is not defined. Please run the cell where combined_df is created.")

# Save the cleaned DataFrame to CSV
combined_df.to_csv(output_file, index=False)

print(f"Cleaned and merged data saved to {output_file}")
# AI-ASSIST: ChatGPT + Copilot (06/08/2025) — draft + refactor; author reviewed and tested.

Cleaned and merged data saved to data/combined_cleaned_final.csv


# Conclusion and next steps

This ETL pipeline successfully extracted data from multiple sources, merged them into a unified dataset, and performed thorough cleaning to handle duplicates and missing values. The resulting dataset is now consistent, complete, and ready for further analysis.

Next Steps:

Feature Engineering: Enhance the dataset by creating new meaningful features that can improve model performance.

Data Encoding: Apply techniques such as one-hot encoding to convert categorical variables into a machine-readable format.

Scaling and Normalisation: Normalise numerical features to prepare for machine learning algorithms.

Modeling and Evaluation: Use the cleaned and engineered data for predictive modeling and evaluate model effectiveness.

Documentation: Continue documenting each step for clarity and reproducibility.

This structured approach will ensure a smooth transition from data preparation to building robust machine learning models.

## Attribution & AI Assistance

Parts of this notebook (section layout, cleaning loops, join/merge patterns, export snippets) were assisted by **ChatGPT** and **GitHub Copilot**.  
All code and outputs were reviewed, modified, and validated by the author. Any mistakes are my own.

- Tools: ChatGPT (structure/markdown/code templates), Copilot (inline suggestions & refactors)  
- Scope: NA handling, type casting, column standardisation, dataset exports  
**Date:** 06 Aug 2025 • Author: Nassra Ibrahim